# Welcome to the Quantum Computing 4 Africa Workshop 

We’re thrilled to have you join us for this exciting and hands-on introduction to quantum computing! Whether you're a student, researcher, developer, or industry profesional, this workshop is designed to give you practical exposure to one of the most revolutionary fields in technology today.

## What will you learn

The following labs will introduce you to the foundations of quantum computing using Python and the Qiskit framework. Throughout this workshop, you'll explore:

- The basics of quantum gates and quantum circuits

- Exploring the bloch sphere [IBM Quantum Composer](https://quantum.cloud.ibm.com/composer)

- How to build and run circuits on simulators and real quantum hardware

- An introduction to quantum machine learning using the sklearn dataset

### Getting Started

The main goal of this introductory lab is to show you how to get started with quantum computing using Python. Quantum computers stand as a promising technology that promises to disrupt how some calculations are done. From breaking encryptation with Shor's algorithm, to enable faster searches with Grover's to even design better batteries with quantum phase estimation. However, when we think about these nice applications, we have to think as well about how to design and execute these algorithms. For many of us this tool is Qiskit! A software for design and implementation of quantum circuits in simulators and real hardware. We will the Qiskit library, which is one of the most popular quantum computing libraries in Python.

you will need Qiskit 2.0 and this notebook is going to provide you with the detailed instructions to get your machine ready to go!

First you must check that the version of python you are using in your environment is python>=3.9.6, to make sure that it will be compatible with the latest Qiskit version we will use

Furthermore, you can also find many useful resources on IBM's new page of quantum education [IBM Quantum Learning](https://learning.quantum.ibm.com/).

In [ ]:
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, transpile, generate_preset_pass_manager
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import SparsePauliOp

from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    SamplerV2 as Sampler,
    EstimatorV2 as Estimator
)

from qiskit_aer import AerSimulator

from qiskit.circuit.library import RealAmplitudes
from qiskit import QuantumRegister

from qiskit_ibm_runtime import Session, SamplerV2 as Sampler

## Sanity check <a id="sanity-check"></a>

Let's now create a very simple quantum circuit to check that everything is working as expected and we are not incurring into any unexpected error.

In [ ]:
# Create a new circuit with a single qubit
qc = QuantumCircuit(2)
# Add a H gate to qubit 0
qc.h(0)
# Add a CNOT gate to qubit 1
qc.cx(0, 1)
# Return a drawing of the circuit using MatPlotLib ("mpl").
qc.draw("mpl")

The drawing you are seeing represents a quantum circuit that produces a Bell state:

$$|Bell\rangle=\frac{|00\rangle+|11\rangle}{\sqrt{2}}$$

#  Exercise 1.1: Hello, Quantum! — Classical Bits vs Quantum Bits 


* **Classical bit**: Either `0` or `1`.
* **Quantum bit (qubit)**: A unit vector in a 2‑D complex Hilbert space $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$ where $|\alpha|^2 + |\beta|^2 = 1$.
* **Bloch sphere**: Geometric representation of a single qubit using spherical coordinates $(\theta,\;\phi)$:

$$|\psi\rangle = \cos\tfrac{\theta}{2}|0\rangle + e^{i\phi}\sin\tfrac{\theta}{2}|1\rangle$$

Changing $\theta$ moves the state between north and south poles (superposition). Changing $\phi$ rotates around the vertical axis (phase).

Changing $\theta$ moves the state between the poles (controls **superposition**). Changing $\phi$ spins the state around the vertical axis (controls **phase**).



#  Exercise 1.1: Hello, Quantum! — Generating Superposition

Superposition is what lets a qubit hold both 0 and 1 at once—fueling quantum parallelism in algorithms from molecular simulation to optimization. Here, we’ll build a 1-qubit circuit, apply a Hadamard (H) gate to create an equal superposition, then measure to verify the 50/50 split.

In [ ]:
# ┌────────────────────────────────────────────────────┐
# │ Create & Measure a Superposition  
# └────────────────────────────────────────────────────┘

# 1. Imports
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# 2. Build the circuit (1 qubit + measurement)
qc = QuantumCircuit(1, 1)
qc.h(0)
qc.measure(0, 0)

# 3. Initialize the Aer simulator
sim = AerSimulator()

# 4. Transpile for the simulator
t_qc = transpile(qc, sim)

# 5. Run the job and collect results
job = sim.run(t_qc, shots=1024)
result = job.result()
counts = result.get_counts()

# 6. Inspect results
print("Measurement counts:", counts)
plot_histogram(counts)
plt.show()


Measurement counts: {'0': 497, '1': 527}


## Interpretation

You should see roughly equal counts for '0' and '1'—that’s your superposition collapsing 50/50 upon measurement.

This lays the groundwork for quantum speedups: the H-gate distributes amplitude across states, enabling parallel exploration of possibilities.

### Next Steps

Vary shots: Change shots=1024 to a smaller or larger number—do the probabilities converge?

Double H: What happens if you apply qc.h(0) twice before measuring? (Hint: H·H = Identity)

# Exercise 1.2: Generate a three-qubit GHZ state using Qiskit patterns <a id="ghz"></a>

Now, we will follow this episode of [Coding with Qiskit](https://www.youtube.com/watch?v=93-zLTppFZw&list=PLOFEBzvs-VvrgHZt3exM_NNiNKtZlHvZi&index=4) to guide you through the process of generating a three-qubit GHZ state using [Qiskit patterns](https://quantum.cloud.ibm.com/docs/en/guides/intro-to-patterns). 

A Qiskit pattern is a general framework for breaking down domain-specific problems and contextualizing required capabilities in stages. This allows for the seamless composability of new capabilities developed by IBM Quantum® researchers (and others) and enables a future in which quantum computing tasks are performed by powerful heterogenous (CPU/GPU/QPU) computing infrastructure. 

The four steps of a Qiskit pattern are as follows:

1. **Map** problem to quantum circuits and operators
2. **Optimize** for target hardware
3. **Execute** on target hardware
4. **Post-process** results


## Step 1. Map <a id="map"></a>

The Greenberger–Horne–Zeilinger (GHZ) state is the extension to three (or more) qubits to the maximally entangled state characteristic of the Bell state depicted above. That means that the GHZ state is:

$$
|GHZ\rangle = \frac{|000\rangle+|111\rangle}{\sqrt{2}}.
$$

One of the interesting things of the GHZ state is that there are different and equivalent ways to build it using a quantum circuit. In Exercise 1 you are asked to do it in one of the most common ones.

<a id="Exercise_1"></a>
<div class="alert alert-block alert-success">
    
<b>Exercise 1: Design a GHZ state</b> 

In this exercise you are asked to design a GHZ state following the steps below:

1. Applying a Hadamard gate to qubit 0, putting it into a superposition. 
2. Apply a CNOT gate between qubits 0 and 1.
3. Apply a CNOT gate between qubits 1 and 2.

</div>

In [2]:
# Create a new circuit with three qubits
qc = QuantumCircuit(3)

### WRITE YOUR CODE BELOW HERE ###

# Add a H gate to qubit 0

# Add a CNOT gate to qubits 0 and 1

# Add a CNOT gate to qubits 1 and 2

### YOUR CODE FINISHES HERE    ###

## Step 2. Optimize and Transpile <a id="optimize"></a>

Well done designing the circuit!

Let us consider, for the GHZ state, a situation in which we are limited to interactions only between qubits 0 and 1 and qubits 0 and 2, but not between qubits 1 and 2. We can introduce these constraints to the transpiler using the `transpile` function and take a look to the resultant circuit.

In [3]:
# from qiskit.visualization import plot_circuit_layout
 
# plot_circuit_layout(qc_t_cm_lv0, backend, view="physical")

## Step 3. Execute <a id="execute"></a>

The next step is exciting, we are going to run the quantum circuit using Qiskit Runtime! 

We will do that using the two [Qiskit Primitives](https://quantum.cloud.ibm.com/docs/en/guides/primitives):
1. Sampler: it samples the output register from the execution of one or more quantum circuits. Its output is counts on per-shot measurements. 
2. Estimator: it computes the expectation value of one or more observables with respect to the states generated by the quantum circuit. Its output consist of the expectation values along with their standard errors.

First, we execute our circuit using the Sampler, then save the results as the variable `results_sampler`. 